In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from mc2.data_management import EXPERIMENT_LOGS_ROOT

In [ ]:
exp_id = "f57e0626"
seed = 1
def load_preds_n_gt(exp_id, seed):
    fp = EXPERIMENT_LOGS_ROOT / f"exp_{exp_id}_seed_{seed}_preds_transformed.parquet"
    fp2 = EXPERIMENT_LOGS_ROOT / f"exp_{exp_id}_seed_{seed}_gt_transformed.parquet"
    fp3 = EXPERIMENT_LOGS_ROOT / f"exp_{exp_id}_seed_{seed}_preds_untransformed.parquet"
    fp4 = EXPERIMENT_LOGS_ROOT / f"exp_{exp_id}_seed_{seed}_gt.parquet"
    preds_transformed_MS = pd.read_parquet(fp).to_numpy()
    preds_untransformed_MS = pd.read_parquet(fp3).to_numpy()
    gt_transformed_MS = pd.read_parquet(fp2).to_numpy()
    gt_MS = pd.read_parquet(fp4).to_numpy()
    return gt_MS, gt_transformed_MS, preds_transformed_MS, preds_untransformed_MS


In [ ]:
experiments_d = {
    "with_transform_h_factor_1.2": {'exp_id': "f57e0626", "seed": "1"},
    "no_h_transform_5_units": {"exp_id": "5fa2797f", "seed": "3"},
    "no_h_transform_3_units": {"exp_id": "ae30dad0", "seed": "4"}
}

In [ ]:

def plot_first_predictions(exp_key):
    gt_MS, gt_transformed_MS, preds_transformed_MS, preds_untransformed_MS = load_preds_n_gt(**experiments_d[exp_key])
    fig, axes = plt.subplots(5, 2, sharex=True, sharey="col", figsize=(20, 15))
    mae_M = np.mean(np.abs(gt_MS - preds_untransformed_MS), axis=-1)
    mse_M = np.mean((gt_MS - preds_untransformed_MS)**2, axis=-1)
    print(f"MAE {mae_M.mean():.1f} A/m | MSE {mse_M.mean():.1f} (A/m)²")
    for tst_idx in range(axes.shape[0]):
        axes[tst_idx, 0].plot(gt_MS[tst_idx], label='gt')
        axes[tst_idx, 0].plot(preds_untransformed_MS[tst_idx], label='pred', ls='dashed')
        axes[tst_idx, 1].plot(gt_transformed_MS[tst_idx])
        axes[tst_idx, 1].plot(preds_transformed_MS[tst_idx], ls='dashed')
        axes[tst_idx, 0].annotate(f"MAE {mae_M[tst_idx]:.1f} A/m | "
                                f"MSE {mse_M[tst_idx]:.1f} (A/m)²",
                                (0.3, 0.1), xycoords=axes[tst_idx, 0].transAxes)
    axes[0, 0].legend()
    for ax in axes.flatten():
        ax.grid(alpha=0.3)
    for ax in axes[:, 0]:
        ax.set_ylabel("H in A/m")
    for ax in axes[:, 1]:
        ax.set_ylabel("H normalized")
    for ax in axes[-1, :]:
        ax.set_xlabel("Sequence step")
    fig.tight_layout()


In [ ]:
plot_first_predictions("no_h_transform_3_units")

In [ ]:
plot_first_predictions("no_h_transform_5_units")

In [ ]:
plot_first_predictions("with_transform_h_factor_1.2")